In [38]:
import codecs
from datetime import timedelta
from transliteration import transliterate

Read CTM file

In [45]:
ctm_file_path = ('/media/ai/storage/kaldi/500_hrs/exp/nnet3/tdnn_reduced'
                 '/decode_test_redhen_sample/1.ctm')
srt_file_path = ('/media/ai/storage/kaldi/500_hrs/exp/nnet3/tdnn_reduced'
                 '/decode_test_redhen_sample/1.srt')
words_file_path = ('/home/ai/Programs/kaldi/egs/500_hrs/s5/data/lang/'
                   'words.txt')
input_format = 'buckwalter'
output_format = 'unicode'

Take groups of words in 2.5 seconds

Print to SRT file

In [25]:
def make_word_map(word_file_path):
    word_map = dict()
    with codecs.open(words_file_path, 'r') as words_file:
        try:
            for line in words_file:
                word, word_num = line.strip().split()
                word_map[word_num] = word
        except Exception:
            print(('Word file is not in specified format. Each line '
                  'should be in the form: <word> <word_number>'))
#             exit(1)
    return word_map

In [26]:
word_map = make_word_map(words_file_path)

In [42]:
def word_nums2words(line, word_map):
    line = ' '.join([word_map[word_num] for word_num in line.split()])
    return line

In [30]:
def format_time(time):
    time_sec_frac = int(round(time - int(time), 3) * 1000)
    time_sec = timedelta(seconds=int(time))
    return str(time_sec) + ',' + str(time_sec_frac).zfill(3)

In [47]:
def print_st(srt_file, index, lines, word_map, input_format=None,
    output_format=None):
    output = str(index + 1) + '\n'
    start_time = format_time(lines[0]['start_time'])
    end_time = format_time(lines[-1]['start_time'] +
                           lines[-1]['duration'])
    output += start_time + ' --> ' + end_time + '\n'
    st = ' '.join([line['word_num'] for line in lines])
    st = word_nums2words(st, word_map)
    if input_format != None and output_format != None:
        st = transliterate(st, input_format, output_format)
    elif input_format == None and output_format != None:
        raise Exception('Input format not specified.')
    elif input_format != None and output_format == None:
        raise Exception('Output format not specified.')
    output += st + '\n\n'
    srt_file.write(output)

In [48]:
duration = 5
start_time = 0
lines = []
index = 0
srt_file = codecs.open(srt_file_path, 'w')
with codecs.open(ctm_file_path, 'r') as ctm_file:
    for line in ctm_file:
        line = line.strip().split()
        line = {'utt_id': line[0], 'channel': line[1],
                'start_time': float(line[2]),
                'duration': float(line[3]), 'word_num': line[4],
                'confidence': float(line[5])}
        if line['start_time'] >= start_time + duration:
            if lines != []:
                print_st(srt_file, index, lines, word_map,
                        input_format, output_format)
                index += 1
            lines = [line]
            start_time += duration
        else:
            lines.append(line)

In [ ]:
len(word_nums)

In [ ]:
ctm_file.close()